In [1]:
import pandas as pd
import numpy as np
import finlab
from finlab import data
from finlab import backtest

import os
import _beowFmt as fm 
import time
finlab.login(open("config.txt", "r").read())

close = data.get('price:收盤價')
vol = data.get('price:成交股數')
sma20 = close.average(20)
# 近一日收盤價創近60日新高
newhigh = close.rolling(60, min_periods=1).max() == close
cond1 = newhigh
# 前30日(不包含今日)有至少一日的收盤價未創新高
cond2 = (newhigh.shift(1) == 0).rolling(30).sum() > 0
# 第30日～第55日前的收盤價有至少一日收盤價創近60日新高
cond3 = (newhigh.shift(30).rolling(25).sum() > 0)
# 前30日～55日的收盤價最高價小於近日收盤價
cond4 = (close.shift(30).rolling(25).max() < close)
# 近日收盤價大於120前收盤價
cond5 = close > close.shift(120)
# 近日收盤價大於60前收盤價
cond6 = close > close.shift(60)

rev = data.get('monthly_revenue:當月營收')
print(rev)
# 近3月平均營收大於12月平均營收
cond7 = rev.average(3) > rev.average(12)
# 近5日平均成交量大於近20日平均成交量
cond8 = vol.average(5) > vol.average(20)

# 近5日平均成交量 > 2000
cond9 = vol.average(5)/1000 > 2000

buy = cond1 & cond2 & cond3 & cond4 & cond5 & cond6 & cond7 & cond8 & cond9
# 月線下彎
sell = sma20 < sma20.shift()

position = pd.DataFrame(np.nan, index=buy.index, columns=buy.columns)
position[buy] = 1
position[sell] = 0
position = position.ffill().fillna(0)


# report = backtest.sim(position.loc['2014':], resample="W",name='二次創高股票', live_performance_start='2018-06-01')

dff = position.iloc[-1:].transpose()
dff["stock_ids"] = dff.index
dff.columns = ["val", "stock_ids"]

dff = dff[ (dff["stock_ids"].str.len() == 4) & (dff["val"] == True) ]
dff = dff.dropna(how='all')

# print(dff)
today, ss = close.iloc[-1:].index[0].strftime("%Y%m%d"), ""

for c in dff["stock_ids"].tolist():
    ss += f"{c}.TW,"
fm.write_LogFile(f"xq_import_二次創高/{today}_二次創高.csv", ss) 


輸入成功!
symbol      000116  000930  000960  0009A0       1101       1102      1103  \
date                                                                         
2005-02-10     NaN     NaN     NaN     NaN  2746766.0  1094966.0  271258.0   
2005-03-10     NaN     NaN     NaN     NaN  1566092.0   663833.0  156962.0   
2005-04-11     NaN     NaN     NaN     NaN  2504657.0   968271.0  234870.0   
2005-05-10     NaN     NaN     NaN     NaN  2418388.0   876647.0  234952.0   
2005-06-10     NaN     NaN     NaN     NaN  2394229.0   934672.0  221134.0   
...            ...     ...     ...     ...        ...        ...       ...   
2023-09-11     NaN     NaN     NaN     NaN  9379408.0  6469836.0  238917.0   
2023-10-11     NaN     NaN     NaN     NaN  8735157.0  6691905.0  232979.0   
2023-11-10     NaN     NaN     NaN     NaN  9371746.0  6706168.0  257343.0   
2023-12-11     NaN     NaN     NaN     NaN  8786909.0  6915817.0  255428.0   
2024-01-10     NaN     NaN     NaN     NaN        NaN     